In [43]:
# introducing dropout
import warnings
warnings.filterwarnings("ignore")

In [44]:
import tensorflow as tf
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [45]:
# load data
digits = load_digits()
X = digits.data
y = digits.target
y = LabelBinarizer().fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3)

In [46]:
# inspect inputs
X_train[0:2]

array([[ 0.,  0.,  2., 14.,  9.,  1.,  0.,  0.,  0.,  1., 12., 12., 11.,
         8.,  0.,  0.,  0.,  4., 14.,  1.,  0., 13.,  3.,  0.,  0.,  8.,
        13.,  0.,  0., 10.,  6.,  0.,  0.,  5., 16.,  1.,  0.,  8.,  9.,
         0.,  0.,  0., 16.,  0.,  0., 11.,  9.,  0.,  0.,  0., 13., 11.,
        10., 15.,  4.,  0.,  0.,  0.,  3., 15., 16.,  5.,  0.,  0.],
       [ 0.,  0.,  7., 14.,  2.,  0.,  0.,  0.,  0.,  0.,  2., 16.,  6.,
         0.,  0.,  0.,  0.,  0.,  2., 15.,  9.,  0.,  0.,  0.,  0.,  0.,
         2., 16., 12.,  0.,  0.,  0.,  0.,  0.,  2., 16., 16.,  5.,  0.,
         0.,  0.,  0.,  0.,  3., 13., 13.,  0.,  0.,  0.,  0.,  6., 12.,
        14., 16., 12.,  5.,  0.,  0.,  5., 16., 16., 16., 16., 15.]])

In [47]:
# inspect target
y_train[0:2]

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]])

In [48]:
def add_layer(inputs, in_size, out_size, layer_name, activation_function=None, ):
    # add one more layer and return the output of this layer
    Weights = tf.Variable(tf.random_normal([in_size, out_size]))
    biases = tf.Variable(tf.zeros([1, out_size]) + 0.1, )
    Wx_plus_b = tf.matmul(inputs, Weights) + biases
    # here to dropout
    Wx_plus_b = tf.nn.dropout(Wx_plus_b, keep_prob)
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b, )
    tf.summary.histogram(layer_name + '/outputs', outputs)
    return outputs

In [49]:
# define placeholder for inputs to network
keep_prob = tf.placeholder(tf.float32)
xs = tf.placeholder(tf.float32, [None, 64])  # 8x8
ys = tf.placeholder(tf.float32, [None, 10])

In [50]:
# add output layer
l1 = add_layer(xs, 64, 50, 'l1', activation_function=tf.nn.tanh)
prediction = add_layer(l1, 50, 10, 'l2', activation_function=tf.nn.softmax)

In [51]:
# the loss between prediction and real data
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction),
                                              reduction_indices=[1]))  # loss
tf.summary.scalar('loss', cross_entropy)

<tf.Tensor 'loss_3:0' shape=() dtype=string>

In [57]:
# training step
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

sess = tf.Session()
merged = tf.summary.merge_all()
# summary writer goes in here
train_writer = tf.summary.FileWriter("logs/train2", sess.graph)
test_writer = tf.summary.FileWriter("logs/test2", sess.graph)

In [58]:
# tf.initialize_all_variables() no long valid from
# 2017-03-02 if using tensorflow >= 0.12
if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:
    init = tf.initialize_all_variables()
else:
    init = tf.global_variables_initializer()
sess.run(init)

In [59]:
def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs: v_xs})
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys})
    return result

In [ ]:
# somehow not working in notebook...
for i in range(500):
    # here to determine the keeping probability
    sess.run(train_step, feed_dict={xs: X_train, ys: y_train, keep_prob: 0.5})
    if i % 50 == 0:
        # record loss
        train_result = sess.run(merged, feed_dict={xs: X_train, ys: y_train, keep_prob: 1})
        test_result = sess.run(merged, feed_dict={xs: X_test, ys: y_test, keep_prob: 1})
        train_writer.add_summary(train_result, i)
        test_writer.add_summary(test_result, i)

In [ ]:
# https://github.com/MorvanZhou/tutorials/blob/master/tensorflowTUT/tf17_dropout/full_code.py